In [ ]:
import numpy as np
import pdb

from meerkat.contrib.eeg import build_stanford_eeg_dp

## Load EEG datapanel

In [ ]:
dp_art = build_stanford_eeg_dp.out(run_id=200)
dp=dp_art.load()
dp.lz[0]

## Slice dp based on metadata

In [ ]:
from domino.slices.eeg import build_slice

dp_train = dp.lz[dp["split"]=="train"]
dp_train_age = build_slice("correlation", dp_train, correlate="age", corr=0.3, n=100, correlate_threshold=1)
